In [1]:
import pyiron_workflow.graph.gui
!npx esbuild ../js/widget.jsx --minify --format=esm --bundle --outdir=../static

⠙⠹⠸⠼
  ../static/widget.js    1.7mb ⚠️
  ../static/widget.css  15.1kb

⚡ Done in 158ms
⠙

In [2]:
import sys

sys.path.insert(0, "/Users/joerg/git_libs/pyiron_core")
# sys.path

In [3]:
from pyiron_workflow import Workflow, as_out_dataclass_node
from pyironflow.wf_extensions import get_nodes, _to_jsonifyable
import pyiron_nodes as pn
import numpy as np
import json
import pyiron_workflow.wf_graph_tools as gt

### Test wf_graph.py

In [4]:
from pyiron_workflow.wf_graph import Graph, GraphNode, GraphEdge, GuiData

In [5]:
lammps = pn.atomistic.engine.lammps.Code()

macro_graph = Graph().get_graph_from_macro(lammps)

w = macro_graph.display()

w

structure:  NotData


ReactFlowWidget()

In [6]:
w.commands = json.dumps({'command': 'AutomatedLayout'})

In [20]:
sub_graph = Graph(label="graph")
sub_graph.add_node( pn.atomistic.structure.build.Bulk(name='Al'), label="structure")
sub_graph.add_node( pn.atomistic.structure.transform.Repeat(), label="repeat")

# TODO: copy the graph to avoid changing the original graph


graph = Graph()
graph.add_node(pn.atomistic.structure.build.Bulk(name="Al"), label="structure")
graph.add_node(pn.atomistic.structure.transform.Repeat(), label="repeat")
graph.add_node(macro_graph, label="graph")
graph.add_edge("structure", "repeat", "structure", "structure")

# TODO: validate consistency of edges!!
if True:
    # graph.add_edge("repeat", "InitLammps", "structure", "structure")
    graph.expand_node("graph")
else:
    pass
graph.add_edge("repeat", "graph", "structure", "structure")
graph.display_gui_nodes()['style']

w = graph.display()

w


ReactFlowWidget()

In [21]:
graph.display_nodes()

Graph: graph


,id,import_path,label,parent_id,level,node,graph,type,expanded
0,Bulk,None,Bulk,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,customNode,False
1,Repeat,None,Repeat,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,customNode,False
2,graph,None,LammpsMacro,None,0,<pyiron_workflow.simple_workflow.Node object a...,<pyiron_workflow.wf_graph.Graph object at 0x15...,graph,True


In [9]:
pyiron_workflow.graph.gui.display_gui_data()

,label,source_labels,target_labels,import_path,target_values,target_types,source_values,source_types
0,Bulk,[structure],"[name, crystalstructure, a, c, c_over_a, u, or...",pyiron_nodes.atomistic.structure.build.Bulk,"[Al, None, None, None, None, None, False, False]","[str, str, float, float, int, float, bool, bool]",[NotData],[builtins.NoneType]
1,Repeat,[structure],"[structure, repeat_scalar]",pyiron_nodes.atomistic.structure.transform.Repeat,"[NotData, 1]","[NonPrimitive, int]",[NotData],[builtins.str]
2,Code,[generic],"[structure, calculator, potential, working_dir]",pyiron_nodes.atomistic.engine.lammps.Code,"[NotData, NotData, None, test2]","[NonPrimitive, None, str, str]",[NotData],[builtins.NoneType]


In [13]:
graph.display_edges()

Graph: graph


,source,target,sourceHandle,targetHandle
0,structure,repeat,structure,structure


In [10]:
xx

NameError: name 'xx' is not defined

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
from pyironflow.reactflow import ReactFlowWidget
import json

w = ReactFlowWidget()

def on_value_change(change):
    print("print command: ", change["new"])
    command, node_name = change["new"].split(":") 
    if command == 'finished':
        print('done')

def work(w):
    data = dict(
    #    label=graph.label,
        nodes=graph._nodes_to_gui(),
        edges=graph._edges_to_gui(),
        graph=graph._graph_to_gui(),
    )
    # print('data: ', json.dumps(data))
    # w.observe(on_value_change, names="commands")
    time.sleep(0.2)
    w.observe(on_value_change, names="commands")
    w.mydata = json.dumps(data)

thread = threading.Thread(target=work, args=(w,))
display(w)
thread.start()

In [ ]:
xx

In [ ]:
graph.display()

In [ ]:
def on_value_change(change):
    print("print command: ", change["new"])
    command, node_name = change["new"].split(":") 
    if command == 'finished':
        print('done')

data = dict(
#    label=graph.label,
    nodes=pyiron_workflow.graph.gui._nodes_to_gui(),
    edges=pyiron_workflow.graph.gui._edges_to_gui(),
    graph=pyiron_workflow.graph.gui._graph_to_gui(),
)
# print('data: ', json.dumps(data))
w.observe(on_value_change, names="commands")
w.mydata = json.dumps(data)

In [ ]:
import asyncio

class YourClassName:
    def __init__(self):
        self.task_finished_event = asyncio.Event()

    def _edges_to_gui(self, remove_none=True):
        edges = []
        for i, (k, v) in enumerate(self.edges.items()):
            edge_dict = v.asdict(remove_none=remove_none)
            edge_dict["id"] = k
            edge_dict["style"] = {"strokeWidth": 2, "stroke": "black"}

            edges.append(edge_dict)

        return edges

    def display(self, update_layout=True):
        from pyironflow.reactflow import ReactFlowWidget
        import json

        w = ReactFlowWidget()

        data = dict(
            label=self.label,
            nodes=self._nodes_to_gui(),
            edges=self._edges_to_gui(),
            graph=self._graph_to_gui(),
        )
        w.mydata = json.dumps(data)
        w.observe(self.on_value_change, names="commands")

        # Set up the callback to handle the completion of the task
        def on_task_finished(change):
            if change['new'] == 'finished':
                print("Task is finished!")
                self.task_finished_event.set()  # Signal that the task is finished
                # Perform any additional actions needed when the task is finished

        # Observe the 'status' traitlet (or any other relevant traitlet)
        w.observe(on_task_finished, names='status')

        return w

    @staticmethod
    def on_value_change(change):
        print("print command: ", change["new"])
        command, node_name = change["new"].split(":")
        if command == 'finished':
            print('done')

    async def wait_for_task(self):
        await self.task_finished_event.wait()
        print("Event has been triggered, task is finished!")

In [ ]:
w.commands

In [ ]:

w.commands = json.dumps({'command': 'AutomatedLayout'})

In [ ]:
import time
# time.sleep(1)

w.graph = json.dumps(pyiron_workflow.graph.gui._graph_to_gui())
# json.loads(w.graph)

In [ ]:
xx

In [ ]:
nodes[0]['parentID']

In [ ]:
pyiron_workflow.graph.gui._graph_to_gui()

In [ ]:
nodes = pyiron_workflow.graph.gui._nodes_to_gui(remove_none=False)
pyiron_workflow.graph.gui._gui_children(nodes[2])

In [ ]:
pyiron_workflow.graph.gui._graph_to_gui()

In [ ]:
graph.display_gui_nodes()

In [ ]:
pyiron_workflow.graph.gui.display_gui_data()

In [ ]:
xx

In [ ]:
from collections import OrderedDict

Graph(nodes=OrderedDict(structure2=graph.nodes['graph'])).display()

In [ ]:
sub_graph = Graph(
    id="Lammps",
    nodes={
        "code": GraphNode(
            id="code", import_path="pyiron_nodes.atomistic.engine.lammps.Code"
        ),
        "potential": GraphNode(
            id="potential", import_path="pyiron_nodes.atomistic.engine.lammps.Potential"
        ),
    },
    edges={
        0: GraphEdge(source="code", target="potential"),
        # 1: dict(source="structure", target="calculator"),
    },
)


graph = Graph()
graph.add_node(
    GraphNode(
        id="structure",
        import_path="pyiron_nodes.atomistic.structure.build.CubicBulkCell",
    )
)
graph.add_node(
    GraphNode(
        id="calculator", import_path="pyiron_nodes.atomistic.engine.lammps.CalcMD"
    )
)

graph.add_node(sub_graph)

graph.add_edge(
    "structure", "calculator", source_handle="structure", target_handle="structure"
)

graph.expand_node("Lammps")
# graph.collapse_node("Lammps")

pyiron_workflow.graph.gui.display_gui_data()

In [ ]:
graph.display_gui_nodes()

In [ ]:
graph.display()

In [ ]:
xx

In [ ]:
pn.atomistic.engine.lammps.InputCalcStatic().run()

In [ ]:
wf = Workflow("Lammps_macro")
wf.structure = pn.atomistic.structure.build.CubicBulkCell("Al", cell_size=1)

wf.calculator = (
    pn.atomistic.engine.lammps.CalcMD()
)  # temperature=300, n_ionic_steps=10_000)
# wf.potential = pn.atomistic.engine.lammps.Potential(
#     structure=wf.structure, name="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1"
# )
wf.lammps = pn.atomistic.engine.lammps.Code(
    structure=wf.structure,
    calculator=wf.calculator,
    potential="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1",  # wf.potential
)

# print(wf.potential.pull())
wf.calculator.run()
# wf.lammps.inputs.structure.value.outputs, wf.lammps.inputs.potential.value.outputs
# wf.lammps.inputs.potential.value.label
# wf.run(debug=True)

In [ ]:
from dataclasses import dataclass
from collections import OrderedDict


def as_dotdict_dataclass(*args, doc_func=None, **kwargs):
    # def wf_data_class(*args, doc_func=None, keys_to_store=None, **kwargs):
    """
    Extension of the python default dataclass to include methods and functionality needed for pyiron_workflows

    :param args: pass to dataclass decorator
    :param doc_func: function from which to copy docstring
    # :param keys_to_store:
    :param kwargs: pass to dataclass decorator
    :return: dataclass like object with enhanced workflow features
    """

    def wrapper(cls):
        cls = dataclass(*args, **kwargs)(cls)

        # Add/modify a variable
        if doc_func is not None:
            cls.__doc__ = doc_func.__doc__

        # Add new methods
        def keys(self):
            return self.__dict__.keys()

        def items(self):
            return [(k, self[k]) for k in self.keys()]

        def asdict(self, remove_none=False):
            if remove_none:
                return {k: self[k] for k in self.keys() if self[k] is not None}
            else:
                return {k: self[k] for k in self.keys()}

        def __getitem__(self, key):
            return self.__dict__[key]

        def __setitem__(self, key, value):
            if key in self.keys():
                self.__dict__[key] = value

        def select(self, keys_to_store=None):
            if keys_to_store is None:
                keys_to_store = self.keys()  # cls._keys_to_store
            return {k: self[k] for k in keys_to_store}

        setattr(cls, "keys", keys)
        setattr(cls, "items", items)
        setattr(cls, "asdict", asdict)
        setattr(cls, "__getitem__", __getitem__)
        setattr(cls, "__setitem__", __setitem__)
        setattr(cls, "select", select)

        return cls

    return wrapper

In [ ]:
from pyiron_workflow import Node
import pandas as pd


@as_dotdict_dataclass()
class GraphNode:
    id: str  # unique identifier
    import_path: str = None  # import path to the node
    label: str = None  # label/name of the node
    parent_id: str = None  # parent node id
    level: int = 0  # level in the graph
    node: Node = None  # node object
    type: str = "node"  # type of node


@as_dotdict_dataclass()
class GuiNode:
    id: str
    data: dict = None
    position: dict = None
    style: dict = None
    targetPosition: str = "left"
    sourcePosition: str = "right"
    type: str = None
    parentId: str = None
    extent: str = None


class Graph:
    def __init__(
        self, id: str = "graph", nodes: dict = None, edges: dict = None, label=None
    ):
        self.id = id
        if label is None:
            label = id
        self.label = label

        self.nodes = nodes if nodes is not None else OrderedDict()
        self.edges = edges if edges is not None else OrderedDict()

    def add_node(self, node: GraphNode):
        self.nodes[node.id] = node

    def add_graph(self, graph: "Graph"):
        self.nodes[graph.label] = GraphNode(
            id=graph.id, node=graph, label=graph.label, type="graph"
        )

    def add_edge(
        self,
        source: str,
        target: str,
        source_handle: str = None,
        target_handle: str = None,
    ):

        n_edges = len(self.edges.keys())
        self.edges[n_edges] = dict(
            source=source,
            source_handle=source_handle,
            target=target,
            target_handle=target_handle,
        )

    def expand_node(self, node: GraphNode):
        if isinstance(node, Graph):
            for k, v in node.nodes.items():
                print(k, v)
                v.parent_id = node.id
                v.level += 1
                self.nodes[k] = v
            for k, v in node.edges.items():
                self.edges[k] = v
        # self.nodes[node.label] = node

    def transpose_dict_of_dicts(self, nested_dict):
        transposed = {}
        for outer_key, inner_dict in nested_dict.items():
            for inner_key, value in inner_dict.items():
                if inner_key not in transposed:
                    transposed[inner_key] = []
                transposed[inner_key].append(value)
        return transposed

    def transpose_list_of_dicts(self, list_of_dicts):
        # Initialize an empty dictionary to hold the transposed data
        transposed_dict = {}

        # Iterate over each dictionary in the list
        for d in list_of_dicts:
            for key, value in d.items():
                if key not in transposed_dict:
                    transposed_dict[key] = []
                transposed_dict[key].append(value)

        return transposed_dict

    @property
    def nodes_dict(self):
        transposed = self.transpose_dict_of_dicts(self.nodes)
        return transposed

    def edges_dict(self):
        transposed = self.transpose_dict_of_dicts(self.edges)
        return transposed

    @property
    def display_nodes(self):
        print(f"Graph: {self.label}")
        df = pd.DataFrame(self.nodes_dict)
        return df

    @property
    def display_edges(self):
        print(f"Graph: {self.label}")
        df = pd.DataFrame(self.edges_dict())
        return df

    def _nodes_to_gui(self, remove_none=True):
        node_width = 100
        style_node = {
            "backgroundColor": "rgba(255, 0, 255, 0.2)",
            "height": 50,
            "width": node_width,
        }
        macro_style_level_1 = {
            "backgroundColor": "rgba(255, 0, 255, 0.2)",
            "height": 250,
            "width": 400,
        }

        macro_style_level_2 = {
            "backgroundColor": "rgba(255, 0, 255, 0.8)",
            "height": 150,
            "width": 270,
        }

        nodes = []
        for i, (k, v) in enumerate(self.nodes.items()):
            node_dict = GuiNode(
                id=k,
                data=dict(
                    label=k,
                ),
                position=dict(x=i * (node_width + 20), y=0),
                style=style_node,
                targetPosition="left",
                sourcePosition="right",
                # type="customNode",
            )
            if v.type == "graph":
                node_dict.data["label"] = v.label
                # node_dict["type"] = "group"
                node_dict["style"] = macro_style_level_1
                node_dict["position"] = dict(x=i * (node_width + 20), y=-10)
            if v.parent_id is not None:
                node_dict.parentId = v.parent_id
                node_dict.extent = "parent"
            nodes.append(node_dict.asdict(remove_none=remove_none))

        return nodes

    def display_gui_nodes(self):
        node_dict = self.transpose_list_of_dicts(self._nodes_to_gui(remove_none=False))
        return pd.DataFrame(node_dict)

    def _edges_to_gui(self):
        edges = []
        for i, (k, v) in enumerate(self.edges.items()):
            edge_dict = dict(
                id=k,
                source=v["source"],
                target=v["target"],
                # sourceHandle=v["source_handle"],
                # targetHandle=v["target_handle"],
                style={"strokeWidth": 2, "stroke": "black"},
            )
            edges.append(edge_dict)

        return edges

    def display(self):
        from pyironflow.reactflow import ReactFlowWidget

        w = ReactFlowWidget()

        w.nodes = json.dumps(self._nodes_to_gui())
        w.edges = json.dumps(self._edges_to_gui())
        return w

In [ ]:
sub_graph = Graph(
    id="Lammps",
    nodes={
        "code": GraphNode(
            id="code", import_path="pyiron_nodes.atomistic.engine.lammps.Code"
        ),
        "potential": GraphNode(
            id="potential", import_path="pyiron_nodes.atomistic.engine.lammps.Potential"
        ),
    },
    edges={
        0: dict(source="code", target="potential"),
        # 1: dict(source="structure", target="calculator"),
    },
)


graph = Graph()
graph.add_node(
    GraphNode(
        id="structure",
        import_path="pyiron_nodes.atomistic.structure.build.CubicBulkCell",
    )
)
graph.add_node(
    GraphNode(
        id="calculator", import_path="pyiron_nodes.atomistic.engine.lammps.CalcMD"
    )
)

graph.add_graph(sub_graph)

graph.add_edge(
    "structure", "calculator", source_handle="structure", target_handle="structure"
)

graph.display_nodes

In [ ]:
graph.expand_node(sub_graph)

graph.display_nodes

In [ ]:
graph.display_edges

In [ ]:
graph.display_gui_nodes()

In [ ]:
graph._nodes_to_gui()

In [ ]:
# from pyironflow.reactflow import ReactFlowWidget

# w = ReactFlowWidget()

# w.nodes = json.dumps(my_nodes)
# w

In [ ]:
graph.display()

In [ ]:
xx

In [ ]:
node = GraphNode(
    id="1", import_path="pyiron_nodes.atomistic.engine.lammps.InputCalcStatic"
)
json_data = json.dumps(node.asdict())
json_data, node.id

In [ ]:
from dataclasses import dataclass


@dataclass
class NodeGraph:
    id: str  # unique identifier
    import_path: str  # import path to the node
    parent_id: str = None  # parent node id
    level: int = 0  # level in the graph

    def to_dict(self):
        return {
            "id": self.id,
            "import_path": self.import_path,
            "parent_id": self.parent_id,
            "level": self.level,
        }


# Example usage
node = NodeGraph(
    id="1", import_path="pyiron_nodes.atomistic.engine.lammps.InputCalcStatic"
)
json_data = json.dumps(node.to_dict())
print(json_data)

In [ ]:
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

# Add nodes with multiple ports represented as attributes
G.add_node("Node1", input_ports=2, output_ports=3)
G.add_node("Node2", input_ports=1, output_ports=2)
G.add_node("Node3", input_ports=1, output_ports=2)

# Add edges
G.add_edge("Node1", "Node2", output_port=1, input_port=1)
G.add_edge("Node1", "Node2", output_port=2, input_port=2)
G.add_edge("Node2", "Node3", output_port=1, input_port=1)
G.add_edge("Node1", "Node3", output_port=1, input_port=1)

# Draw the graph
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=700,
    node_color="skyblue",
    font_size=10,
    font_weight="bold",
    edge_color="gray",
)

# Display the graph
plt.show()

In [ ]:
# Create a subgraph from the existing graph G
sub_nodes = ["Node1", "Node2"]
subgraph = G.subgraph(sub_nodes)

# Draw the subgraph
pos_sub = nx.spring_layout(subgraph)  # positions for all nodes
nx.draw(
    subgraph,
    pos_sub,
    with_labels=True,
    node_size=700,
    node_color="lightgreen",
    font_size=10,
    font_weight="bold",
    edge_color="gray",
)

# Display the subgraph
plt.show()

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=["id", "import_path"])
df = df.add(
    {"id": "1", "import_path": "pyiron_nodes.atomistic.engine.lammps.InputCalcStatic"}
)
# ({"id": "1", "import_path": "pyiron_nodes.atomistic.engine.lammps.InputCalcStatic"}, ignore_index=True)
# df = df.concat({"id": "2", "import_path": "pyiron_nodes.atomistic.engine.lammps.CalcMD"}, ignore_index=True)
df.c

In [ ]:
import pandas as pd

# Existing DataFrame
df = pd.DataFrame({"Name": ["Alice", "Bob"], "Age": [25, 30]})

# New row as a dictionary
new_row = {"Name": "Charlie", "Age": 35}

# Append the new row
df = df.append(new_row, ignore_index=True)
print(df)

In [ ]:
wf_macro = gt.get_graph_from_wf(
    wf.lammps._wf_macro
)  # connected var_nodes are set to None rather than to node.outputs!
wf_macro.nodes

In [ ]:
wf_macro.edges

In [ ]:
wf.lammps.inputs

In [ ]:
wf.lammps._wf_macro.InitLammps.inputs

In [ ]:
wf = Workflow("Lammps")
wf.structure = pn.atomistic.structure.build.Bulk("Al", cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(
    structure=wf.structure, repeat_scalar=3
)

wf.calculator = (
    pn.atomistic.engine.lammps.CalcMD()
)  # temperature=300, n_ionic_steps=10_000)
wf.potential = pn.atomistic.engine.lammps.Potential(
    structure=wf.structure, name="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1"
)

wf.init_lammps = pn.atomistic.engine.lammps.InitLammps(
    structure=wf.repeat,
    potential=wf.potential,
    calculator=wf.calculator,
    working_directory="test2",
)

wf.shell = pn.atomistic.engine.lammps.Shell(
    # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
    working_directory=wf.init_lammps,
)

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(log_file=wf.shell.outputs.log)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",
)

wf.get_energy_pot = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.Collect)

# nodes_json = get_nodes(wf)
# for i, node in enumerate(nodes_json):
#     print(i, ':', node['id'])
#     print(json.dumps(node))


out = wf.run(debug=True)
out

In [ ]:
len(gt.get_graph_from_wf(wf).edges), gt._load_graph(
    "test_wf.json", workflow_dir="../pyiron_nodes/local_workflows"
).edges

In [ ]:
wf.save("test_wf.json", overwrite=True)

In [ ]:
wf_new = Workflow.load("test_wf.json")

In [ ]:
wf_new.run()

In [ ]:
print(gt.get_code_from_wf(wf_new))

In [ ]:
xx

In [ ]:
nodes = gt.get_nodes_from_wf(
    wf,
    keys_to_keep=[
        "data/label",
        "data/import_path",
        "data/target_values",
        "data/target_labels",
    ],
)
len(nodes)

In [ ]:
wf.structure.inputs

In [ ]:
wf.structure.n_out_labels

In [ ]:
wf_graph = gt.get_graph_from_wf(wf)

In [ ]:
len(wf_graph.edges)

In [ ]:
print(gt.get_code_from_graph(wf_graph))

In [ ]:
gt._save_graph(
    wf_graph, "test.py", workflow_dir="../pyiron_nodes/local_workflows", overwrite=True
)

In [ ]:
graph_loaded = gt._load_graph(
    "test_wf.json", workflow_dir="../pyiron_nodes/local_workflows"
)
print(gt.get_wf_from_graph(graph_loaded))

In [ ]:
len(graph_loaded.edges)

In [ ]:
gt.get_wf_from_graph(graph_loaded)

In [ ]:
print(gt.get_code_from_graph(wf_graph))

In [ ]:
from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("Lammps")

wf.calculator = pyiron_nodes.atomistic.engine.lammps.CalcMD()
wf.structure = pyiron_nodes.atomistic.structure.build.Bulk(name="Al", cubic=True)
wf.repeat = pyiron_nodes.atomistic.structure.transform.Repeat(
    structure=wf.structure.outputs.structure, repeat_scalar=3
)
wf.potential = pyiron_nodes.atomistic.engine.lammps.Potential(
    structure=wf.structure.outputs.structure,
    name="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1",
)
wf.init_lammps = pyiron_nodes.atomistic.engine.lammps.InitLammps(
    structure=wf.repeat.outputs.structure,
    potential=wf.potential.outputs.pot,
    calculator=wf.calculator.outputs.calculator,
    working_directory="test2",
)
wf.shell = pyiron_nodes.atomistic.engine.lammps.Shell(
    working_directory=wf.init_lammps.outputs.path
)
wf.ParseLogFile = pyiron_nodes.atomistic.engine.lammps.ParseLogFile(
    log_file=wf.shell.outputs.log
)
wf.ParseDumpFile = pyiron_nodes.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pyiron_nodes.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",
)
wf.get_energy_pot = pyiron_nodes.atomistic.engine.lammps.GetEnergyPot(
    generic=wf.Collect.outputs.generic
)

wf.run()

In [ ]:
wf_graph.edges

In [ ]:
wf_new = gt.get_wf_from_graph(wf_graph)
wf_new.repeat.pull()

In [ ]:
print(gt.get_code_from_wf(wf_new))

In [ ]:
from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("Lammps")

wf.calculator = pyiron_nodes.atomistic.engine.lammps.CalcMD()
wf.structure = pyiron_nodes.atomistic.structure.build.Bulk(name="Al", cubic="True")
wf.repeat = pyiron_nodes.atomistic.structure.transform.Repeat(
    structure=wf.structure.outputs.structure, repeat_scalar=3
)
wf.potential = pyiron_nodes.atomistic.engine.lammps.Potential(
    structure=wf.structure.outputs.structure,
    name="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1",
)
wf.init_lammps = pyiron_nodes.atomistic.engine.lammps.InitLammps(
    structure=wf.repeat.outputs.structure,
    potential=wf.potential.outputs.pot,
    calculator=wf.calculator.outputs.calculator,
    working_directory="test2",
)
wf.shell = pyiron_nodes.atomistic.engine.lammps.Shell(
    working_directory=wf.init_lammps.outputs.path
)
wf.ParseLogFile = pyiron_nodes.atomistic.engine.lammps.ParseLogFile(
    log_file=wf.shell.outputs.log
)
wf.ParseDumpFile = pyiron_nodes.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pyiron_nodes.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",
)
wf.get_energy_pot = pyiron_nodes.atomistic.engine.lammps.GetEnergyPot(
    generic=wf.Collect.outputs.generic
)

wf.run()

In [ ]:
xx

In [ ]:
from typing import List, Tuple


def graph_edges_to_wf_edges(graph_edges: List[Tuple[str, str]]):
    wf_edges = []
    for edge in graph_edges:
        edge_source, edge_target = edge
        target, target_handle = edge_target.split("/")
        source, source_handle = edge_source.split("/")
        if not source.startswith("var_"):
            edge_dict = dict(
                source=source,
                sourceHandle=source_handle,
                target=target,
                targetHandle=target_handle,
            )
        wf_edges.append(edge_dict)
    return wf_edges

In [ ]:
from pyironflow.wf_extensions import get_node_from_path

wf = Workflow(wf_graph.label)
# Add nodes to Workflow
for node in wf_graph.nodes:
    id, import_path = node

    if not id.startswith("var_"):
        kwargs = dict()

        # Add non-default arguments to node
        first_edge = True
        for edge in wf_graph.edges:
            edge_source, edge_target = edge
            target, target_handle = edge_target.split("/")
            source, source_handle = edge_source.split("/")
            if target == id:
                if source.startswith("var_"):
                    if source_handle.startswith("__str_"):
                        kwargs[target_handle] = source_handle[6:]
                    else:
                        kwargs[target_handle] = source_handle

        new_node = get_node_from_path(import_path)(**kwargs)
        wf.add_node(id, new_node)
wf._set_edges(graph_edges_to_wf_edges(wf_graph.edges))

In [ ]:
print(gt.get_code_from_wf(wf))

In [ ]:
wf_graph.edges

In [ ]:
gt._find_input_nodes(wf_graph, 9)

In [ ]:
gt.get_graph_from_wf(wf).edges

In [ ]:
wf.repeat.pull()

In [ ]:
wf.remove_node("repeat")

In [ ]:
wf._nodes

In [ ]:
wf._edges

In [ ]:
xx

In [ ]:
from dataclasses import dataclass, field, is_dataclass

node = pn.atomistic.calculator.data.OutputCalcMD()
print(
    is_dataclass(node.dataclass())
)  # is already decorated with as_out_dataclass_node -> why? should return only the dataclass

dc = node.dataclass(energies_pot=np.ones(10), energies_kin=2 * np.ones(10))
print(dc)
dc.temperatures = 3 * np.ones(10)
print("dc: ", dc)


node.run()

In [ ]:
out_node = pn.atomistic.calculator.data.OutputCalcMD(dataclass=dc)
out_node.run()
out_node.inputs.data["value"][0]

In [ ]:
wf_out = Workflow("test")
wf_out.add_node("md_output", out_node)
wf_out.run()

In [ ]:
import json

json.dumps(get_nodes(wf_out))

In [ ]:
import json


def remove_non_primitive_values(outputs):
    types = outputs.data["type"]
    values = [
        v if t != "NonPrimitive" else "NotData"
        for v, t in zip(outputs.data["value"], types)
    ]
    return values


remove_non_primitive_values(out_node.outputs)

In [ ]:
node.outputs

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD().dataclass(
    energies_pot=np.ones(10), energies_kin=2 * np.ones(10)
)  # it would be nice to get the dataclass without ceating first an instance of the node

In [ ]:
# wishlist:
# - get the dataclass without creating an instance of the node, e.g.
# - get the dataclass rather than a node when providing as input keys of the dataclass,
#    e.g. pn.atomistic.calculator.data.OutputCalcMD(energies_pot=np.ones(10), energies_kin=2*np.ones(10)) -> dataclass

In [ ]:
out_node.inputs

In [ ]:
node = pn.atomistic.calculator.data.OutputCalcMD()
node._func, node._run, node.kwargs

In [ ]:
xx

In [ ]:
wf = Workflow("Lammps")
wf.structure = pn.atomistic.structure.build.Bulk("Al", cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(
    structure=wf.structure, repeat_scalar=3
)

wf.calculator = (
    pn.atomistic.engine.lammps.CalcMD()
)  # temperature=300, n_ionic_steps=10_000)
wf.potential = pn.atomistic.engine.lammps.Potential(
    structure=wf.structure, name="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1"
)

wf.init_lammps = pn.atomistic.engine.lammps.InitLammps(
    structure=wf.repeat,
    potential=wf.potential,
    calculator=wf.calculator,
    working_directory="test2",
)

wf.shell = pn.atomistic.engine.lammps.Shell(
    # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
    working_directory=wf.init_lammps,
)

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(log_file=wf.shell.outputs.log)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",
)

wf.get_energy_pot = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.Collect)

# nodes_json = get_nodes(wf)
# for i, node in enumerate(nodes_json):
#     print(i, ':', node['id'])
#     print(json.dumps(node))


out = wf.run()
out

In [ ]:
nodes_json = get_nodes(wf)
for k, v in wf._nodes.items():
    # print(k, [type(vv) for vv in v.inputs])#
    print(k, [type(vv) for vv in v.inputs.data["value"]])

In [ ]:
for i in range(0, len(nodes_json)):
    print(i, nodes_json[i]["data"]["target_values"])
    print(i, nodes_json[i]["data"]["target_types"])

In [ ]:
nodes_json[4]["data"]

In [ ]:
wf._nodes["shell"].outputs.__getattr__("log")

In [ ]:
pn.atomistic.calculator.data.InputCalcMD()

In [ ]:
pn.atomistic.calculator.data.InputCalcMD??

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD().run()

In [ ]:
kwargs = pn.atomistic.calculator.data.OutputCalcMD().kwargs
kwargs
pn.atomistic.calculator.data.OutputCalcMD()._func(kwargs)

In [ ]:
len(pn.atomistic.calculator.data.OutputCalcMD().kwargs.keys())

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD.__annotations__

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD()

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD.__annotations__

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD??

Calling the same node a second time changes func_dataclass: becomes a dataclass! Make sure that the decorators act as pure functions!!! Change the instance but not the class!

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD()  # .dataclass()

In [ ]:
wf.calculator.run()

In [ ]:
wf.collect.outputs

In [ ]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node
import pyiron_nodes as pn

wf = Workflow("test")
wf.add_node("out", pn.atomistic.calculator.data.OutputCalcMD())
wf.add_node("out", pn.atomistic.calculator.data.OutputCalcMD())
wf.add_node("out", pn.atomistic.calculator.data.OutputCalcMD())

wf.child_labels

In [ ]:
wf.out.function

In [ ]:
wf.out.to_dict()

In [ ]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node
import pyiron_nodes as pn

wf = Workflow("test")
wf.add_node("inp", pn.atomistic.calculator.data.InputCalcMD())
wf.add_node("inp", pn.atomistic.calculator.data.InputCalcMD())
wf.add_node("inp", pn.atomistic.calculator.data.InputCalcMD())

wf.child_labels

In [ ]:
wf = Workflow("test")
wf.inp = pn.atomistic.calculator.data.InputCalcMD()
wf.out = pn.atomistic.calculator.data.OutputCalcMD(dataclass=wf.inp)
wf.run()

In [ ]:
wf.out.to_dict()

In [ ]:
wf.inp.to_dict()

In [ ]:
pn.atomistic.calculator.data.OutputCalcMD().kwargs

In [ ]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node
import pyiron_nodes as pn

wf = Workflow("test")
wf.add_node("bulk", pn.atomistic.structure.build.Bulk())
wf.add_node("bulk", pn.atomistic.structure.build.Bulk())
wf.add_node("bulk", pn.atomistic.structure.build.Bulk())

wf.child_labels

In [ ]:
wf.bulk_1.to_dict()

In [ ]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node
import pyiron_nodes as pn
from pyironflow.wf_extensions import get_nodes
import json

wf = Workflow("test")
wf.inp_calc = pn.atomistic.calculator.data.InputCalcMD()
json.dumps(get_nodes(wf))

In [ ]:
pn.atomistic.structure.build.Bulk().inputs

In [ ]:
xx

In [ ]:
wf = Workflow("Lammps")
wf.structure = pn.atomistic.structure.build.Bulk("Al", cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(
    structure=wf.structure, repeat_scalar=3
)

wf.calculator = (
    pn.atomistic.engine.lammps.CalcMD()
)  # temperature=300, n_ionic_steps=10_000)
wf.potential = pn.atomistic.engine.lammps.Potential(
    structure=wf.structure, name="1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1"
)

wf.init_lammps = pn.atomistic.engine.lammps.InitLammps(
    structure=wf.repeat,
    potential=wf.potential,
    calculator=wf.calculator,
    working_directory="test2",
)

wf.shell = pn.atomistic.engine.lammps.Shell(
    # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
    working_directory=wf.init_lammps,
)

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(log_file=wf.shell.outputs.log)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",
)

wf.get_energy_pot = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.Collect)

out = wf.run()

In [ ]:
import pyiron_nodes
import pyiron_nodes.atomistic

pyiron_nodes.atomistic.structure.build.CubicBulkCell("Al")

In [ ]:
from ase import Atoms as _Atoms
import numpy as _np
from typing import Optional


@as_function_node("plot")
def Plot3d(
    structure: _Atoms,
    camera: str = "orthographic",
    particle_size: Optional[int | float] = 1.0,
    select_atoms: Optional[_np.ndarray] = None,
    view_plane: _np.ndarray = _np.array([0, 0, 1]),
    distance_from_camera: Optional[int | float] = 1.0,
):
    """Display atomistic structure (ase.Atoms) using nglview"""
    from structuretoolkit import plot3d

    return structure.plot3d(
        # structure=structure,
        camera=camera,
        particle_size=particle_size,
        select_atoms=select_atoms,
        view_plane=view_plane,
        distance_from_camera=distance_from_camera,
    )

In [ ]:
Plot3d().inputs

In [ ]:
import numpy as np
from pyiron_workflow import Workflow, as_function_node, as_macro_node


@as_function_node
def linspace(x_min: int = 0, x_max: float = 2 * np.pi, n: int = 50):

    vec = np.linspace(x_min, x_max, n)
    return vec


@as_function_node(labels=["vec"])
def sin(x):
    import numpy as np

    vec = np.sin(x)
    return vec


@as_function_node("plot")
def plot(x, y):
    import matplotlib.pylab as plt

    plt.plot(x, y)
    return plt.show()


@as_macro_node(labels="structure")
def bulk_macro(element: str, cell_size: int = 1, vacancy_index: int = 0):
    from pyiron_nodes.atomistic.structure.transform import (
        CreateVacancy,
        Repeat,
    )
    from pyiron_nodes.atomistic.structure.build import Bulk

    wf = Workflow("bulk_macro")
    wf.bulk = Bulk(name=element, cubic=True)
    wf.cell = Repeat(structure=wf.bulk, repeat_scalar=cell_size)

    wf.structure = CreateVacancy(structure=wf.cell, index=vacancy_index)
    return wf.structure

In [ ]:
from pyiron_nodes.atomistic.structure.view import Plot3d

wf = Workflow("simple_workflow")
wf.linspace = linspace(n=100)
wf.sin = sin(x=wf.linspace)
wf.plot = plot(x=wf.linspace, y=wf.sin)

wf.bulk_macro = bulk_macro(element="Al", cell_size=1, vacancy_index=0)

wf.run()

In [ ]:
wf.bulk_macro._wf_macro._edges

In [ ]:
dir(wf.bulk_macro)

In [ ]:
wf.bulk_macro.inputs.cell_size = 3
wf.bulk_macro.pull()  # does not work

In [ ]:
wf.sin(x=[2, 3])

In [ ]:
wf.sin.inputs.x = [2, 3, 4]
wf.sin.run()

In [ ]:
wf.bulk_macro.run()
wf.bulk_macro.outputs

In [ ]:
from pyiron_nodes.atomistic.structure.view import Plot3d

wf = Workflow("simple_workflow")
# wf.linspace = linspace(n=100)
# wf.sin = sin(x=wf.linspace)
# wf.plot = plot(x=wf.linspace, y=wf.sin)

wf.bulk_macro = bulk_macro(element="Al", cell_size=1, vacancy_index=0)
wf.plot3d = Plot3d(structure=wf.bulk_macro)

wf.run()

In [ ]:
out = wf._nodes["bulk_macro"].run()
out

In [ ]:
wf.bulk_macro.run().plot3d()

In [ ]:
wf.plot3d.inputs

In [ ]:
Plot3d(out).run()